<a href="https://colab.research.google.com/github/pedromazim/visao/blob/main/project_fractal_dimension_texture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### I. Modules and auxiliary functions

In [3]:
import numpy as np
import skimage as skm
import matplotlib.pyplot as pt
import matplotlib.colors as cm
#import porespy as ps
from numba import jit
from PIL import Image
import os
from glob import glob
from skimage.filters import threshold_otsu

In [4]:
def make_gray(img, weights):

    '''
    converts an colored RGB image to grayscale

    img : image to be converted
    weights : list or array containing the relative weights of red, blue and green for conversion

    '''

    if len(img.shape) == 3:

        img_recol = np.zeros((img.shape[0],img.shape[1]))

        # for i in range(img.shape[0]):
        #     for j in range(img.shape[1]):
        #             img_recol[i,j] = weights[0]*img[i,j,0] + weights[1]*img[i,j,1] + weights[2]*img[i,j,2]
        img_recol = weights[0]*img[:,:,0] + weights[1]*img[:,:,1] + weights[2]*img[:,:,2]
        return img_recol
    else:
        return img

In [5]:
@jit(looplift=True)
def histogram(img):

    '''
    returns the histogram of a grayscale image

    img: input image
    '''

    bins = np.arange(256)
    counts = np.zeros_like(bins)

    for b in bins:
        counts[b] = np.where(img==b)[0].size

    return counts

<ipython-input-5-8337ce144aa7>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(looplift=True)


In [6]:
@jit(looplift=True)
def cdf_histogram(img):

    '''
    returns the cummulative histogram of a grayscale image

    img: input image
    '''

    bins = np.arange(256)
    counts = histogram(img)
    cdf_counts = np.zeros_like(counts)

    for b in bins:
        cdf_counts[b] = np.sum(counts[0:b])/(img.size)

    return cdf_counts


<ipython-input-6-bddc8ff98c4a>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(looplift=True)


In [7]:
#@jit(looplift=True)
def otsu_thresholding(img):
    img = img.flatten()
    hist = histogram(img)
    cdf = cdf_histogram(hist)
    mean_gray = np.mean(img)
    var = np.zeros_like(hist)

    for t in range(256):
        # computing class probability (background and foreground)
        pb = img[img<t].size
        wb = pb/img.size
        wf = 1-wb

        if wb != 0 and wb != 1:
            mb = np.sum(img[img<t])/pb
            mf = np.sum(img[img>=t])/(img.size-pb)

            vb = np.sum((img[img<t]-mb)**2)/pb
            vf = np.sum((img[img>=t]-mf)**2)/(img.size-pb)

        elif wb == 1:
            continue
        else:
            continue

        var[t] = wb*vb + wf*vf
    count = 0
    for v in var:
        if v != 0:
            break
        else:
            count+=1
        #print(wb,vb, wf,vf, var[t])
    threshold = np.argmin(var[np.nonzero(var)]) + count

    return threshold

In [8]:
# ROUBALHEIRA PESADA
@jit(looplift=True)
def find_divisors(num_a,num_b):# finds common divisors :)
    d_a, d_b = [],[]
    for i in range(2,int(num_a/2)+1):
        if num_a%i == 0:
            d_a.append(i)
    for i in range(2,int(num_b/2)+1):
        if num_b%i == 0:
            d_b.append(i)
    d = np.intersect1d(d_a,d_b)
    return np.array(d)

<ipython-input-8-00525284bd13>:2: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(looplift=True)


In [9]:
@jit(looplift=True)
def edt(img):# euclidean distance from foreground to background pixels

    img = img.astype('int')
    h, w = img.shape

    d = np.zeros((h,w))

    for y in range(h):
        for x in range(w):

            if img[x,y] == 1:
                flag = True
                r = 1
                while flag:
                    nb = img[x-r:x+r+1,y-r:y+r+1]
                    if np.flatnonzero(nb).size != nb.size:
                        d[x,y] = r
                        flag = False
                    else:
                        r += 1
    return d

<ipython-input-9-3228bbd79bae>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(looplift=True)


In [24]:
@jit(looplift=True)
def distance(img, x0, y0):# distance from all foreground pixels to a specific one
    img = img.astype('int')
    h, w = img.shape
    dist = np.zeros_like(img)
    fg = np.argwhere(img == 1)
    dist = np.sqrt((np.indices((h,w))[0]-x0)**2 + (np.indices((h,w))[1]-y0)**2)
    # for px in fg:
    #     dist[x,y] = np.sqrt((px[0]-x0)**2+(px[1]-y0)**2)
    return dist

<ipython-input-24-c6de0aadb2f4>:1: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @jit(looplift=True)


### II. Box-counting method

Input is a binary image\
Step 1: Cut the image in four squares (cut in half in width and height directions)\
Step 2: Count how many squares have at least one foreground pixel in it\
Step 3: Divide each square again and count


In [ ]:
#@jit(looplift=True)
def bcm(img):

    img = make_gray(img,[1/3,1/3,1/3])
    #thd = otsu_thresholding(img)
    thd = threshold_otsu(img)
    img = img > thd

    img = img.astype('int')
    grid = np.zeros_like(img)
    h, w = img.shape
    counts = []

    # for i in range(1,res):# divide squares each loop
    #     L = 2**(i)
    #     counter = 0
    #     for s in np.split(img,L):
    #         s = np.split(s,L,axis=1)
    #         for square in s:
    #             if np.flatnonzero(square).size != 0:
    #                 counter += 1
    #     counts.append(counter)

    counter = 0

    for d in find_divisors(h,w):
        counter = 0
        s = img.reshape(d, int(h/d), d, int(w/d)).transpose(0,2,1,3)
        # print(s.shape)
        for ss in s:
            for square in ss:
                # print(square)
                if np.flatnonzero(square).size != 0:
                    counter += 1
        counts.append(counter)
    return np.array(counts)

### III. Bouligand-Minkowsky method
Input is a binary image\
Step 1: Obtain the edge pixels by euclidean distance transform (EDT)\
Step 2: Set a disk of radius r and count every pixel under its influence area;\
Step 3: Repeat for every edge pixel;\
Step 4: Repeat for many r values;\
Step 5: Obtain FD by linearization of A(r) x r.

In [26]:
def bmm(img, R):

    img = make_gray(img,[1/3,1/3,1/3])
    #thd = otsu_thresholding(img)
    thd = threshold_otsu(img)
    img = img > thd
    img = img.astype('int')

    dt = edt(img)
    edge = np.argwhere(dt==1)
    counts = []
    for r in range(1,R+1):# calculates influence area for multiple r values
        print(r)
        c = 0
        for px in edge:
            A_r = np.flatnonzero(np.where(distance(img, px[0],px[1]) <= r))
            c += A_r.size
        counts.append(c)

    return counts

In [20]:
img = skm.io.imread(f'https://github.com/pedromazim/visao/blob/main/leaf_data/{filename_list[50]}?raw=true')[:,1000:-1000,:]

In [21]:
img = make_gray(img,[1/3,1/3,1/3])
#thd = otsu_thresholding(img)
thd = threshold_otsu(img)
img = img > thd
img = img.astype('uint8')

In [ ]:
test = bmm(img,10)

### IV. Applying to a leaf image dataset

In [27]:
!git clone https://github.com/pedromazim/visao.git

fatal: destination path 'visao' already exists and is not an empty directory.


In [28]:
%cd visao/leaf_data/
os.getcwd()

/content/visao/leaf_data


'/content/visao/leaf_data'

In [29]:
filename_list = glob("*.JPG")

In [30]:
#@jit(looplift=True)
def fd_dataset(filename_list): #calculates BCM and BMM for each image in dataset

    # BCM_fd = np.empty((22,len(filename_list)))
    # BMM_fd = np.empty((,len(filename_list)))
    BMM_fd = []
    for filename in range(len(filename_list)):
        #print(filename)
        img = skm.io.imread(f'https://github.com/pedromazim/visao/blob/main/leaf_data/{filename_list[filename]}?raw=true')[:,1000:-1000,:]

        # bcm_fd = bcm(img)
        # print(filename, 'bcm')
        bmm_fd = bmm(img, iter)
        print(filename, 'bmm')

        BCM_fd[:,filename] = bcm_fd
        BMM_fd.append(bmm_fd)

    data = np.savetxt('out_leaf.dat', np.array(BMM_fd))
    return data

In [ ]:
out_test_bmm = fd_dataset(filename_list)

In [16]:
data_bcm = np.loadtxt('out_leaf.dat')

In [17]:
coeffs = []
for j in range(data_bcm.shape[1]):
    im_ex = skm.io.imread(f'https://github.com/pedromazim/visao/blob/main/leaf_data/{filename_list[j]}?raw=true')[:,1000:-1000,:]
    sizes = find_divisors(im_ex.shape[0],im_ex.shape[1])
    coeffs.append(np.polyfit(np.log(sizes),np.log(data_bcm[:,j]),1))
coeffs = np.array(coeffs)

In [18]:
coeffs

array([[ 1.80615496e+00,  1.25433867e-01],
       [ 1.70132467e+00,  5.94026410e-01],
       [ 1.88606992e+00,  8.81843167e-02],
       [ 1.59663217e+00,  5.20962666e-01],
       [ 1.83604836e+00,  2.32445777e-01],
       [ 1.90238642e+00,  1.75532733e-01],
       [ 1.93506202e+00,  3.43774320e-02],
       [ 1.43843797e+00,  8.61317068e-01],
       [ 1.94958363e+00,  3.26064443e-03],
       [ 1.66494391e+00,  3.41840528e-02],
       [ 1.91704492e+00,  1.25914555e-01],
       [ 1.62738822e+00,  8.13817952e-01],
       [ 1.94588620e+00,  1.78443624e-02],
       [ 1.93774357e+00,  2.67740374e-02],
       [ 1.82992583e+00,  3.06098138e-01],
       [ 1.77879909e+00,  4.14099521e-01],
       [ 1.64391017e+00,  1.23176343e-01],
       [ 1.41583820e+00,  5.65725405e-01],
       [ 1.70080021e+00,  5.89690626e-01],
       [ 1.60066811e+00,  5.46601035e-01],
       [ 1.88672990e+00,  1.23707038e-01],
       [ 1.49074604e+00,  4.09834900e-01],
       [ 1.95416165e+00,  3.53499713e-02],
       [ 1.

In [20]:
data_bcm[:,1].size

19

In [22]:
find_divisors(4000,4000).size

22

In [87]:
bcm(im_ex)

array([      4,      15,      24,      54,      78,     192,     287,
           427,     668,    1006,    1501,    3540,    5316,    8005,
         12626,   19104,   28885,   69941,  106660,  264974,  409104,
       1595392])

In [21]:
a = np.array([1,2,3])
b = np.arange(9).reshape(3,3)
np.indices((3,3))[0]

array([[0, 0, 0],
       [1, 1, 1],
       [2, 2, 2]])

In [25]:
distance(b,1,1)

<ipython-input-24-c6de0aadb2f4>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "distance" failed type inference due to: Use of unsupported NumPy function 'numpy.indices' or unsupported use of the function.

File "<ipython-input-24-c6de0aadb2f4>", line 7:
def distance(img, x0, y0):# distance from all foreground pixels to a specific one
    <source elided>
    fg = np.argwhere(img == 1)
    dist = np.sqrt((np.indices((h,w))[0]-x0)**2 + (np.indices((h,w))[1]-y0)**2)
    ^

During: typing of get attribute at <ipython-input-24-c6de0aadb2f4> (7)

File "<ipython-input-24-c6de0aadb2f4>", line 7:
def distance(img, x0, y0):# distance from all foreground pixels to a specific one
    <source elided>
    fg = np.argwhere(img == 1)
    dist = np.sqrt((np.indices((h,w))[0]-x0)**2 + (np.indices((h,w))[1]-y0)**2)
    ^

  @jit(looplift=True)
/usr/local/lib/python3.10/dist-packages/numba/core/object_mode_passes.py:151: NumbaWarning: Function "dista

array([[1.41421356, 1.        , 1.41421356],
       [1.        , 0.        , 1.        ],
       [1.41421356, 1.        , 1.41421356]])